# Inspectioning the desnmaps files

In [ ]:
from astropy.io import fits
import pandas as pd
import numpy as np

# Path to the file (adjust as needed)
path = "LSST_DP1/densmap_o5.fits"  # for example, order 5

# Open the FITS file
with fits.open(path) as hdul:
    # Overview of the HDUs
    hdul.info()

    # Header of the binary table (where the data is stored)
    print("\nHeader of HDU 1:")
    print(hdul[1].header)

    # Convert the binary table (HDU 1) to a pandas DataFrame
    df = pd.DataFrame(np.array(hdul[1].data))

# Display the first few rows as a DataFrame
print("\nFirst rows of densmap table:")
display(df.head(10))

# Extract the HEALPix count values
values = df["VALUE"].to_numpy()

# Print basic statistics
print(f"\nNumber of pixels: {len(values):,}")
print(f"Total number of sources: {np.sum(values):,}")
print(f"Min / Max counts per pixel: {values.min()} / {values.max()}")

In [ ]:
import healpy as hp
import matplotlib.pyplot as plt
from astropy.io import fits

# Load the HEALPix density map (VALUE column)
values = fits.getdata("LSST_DP1/densmap_o5.fits", ext=1)["VALUE"]

# Infer nside automatically from the number of pixels
nside = hp.npix2nside(len(values))
print(f"nside: {nside}, total pixels: {len(values)}")

# Display the density map using a Mollweide projection
hp.mollview(values, nest=True, title="LSST_DP1 densmap (order 5)")
hp.graticule()
plt.show()

# Inspectioning the MOC files

In [ ]:
from astropy.io import fits
import pandas as pd
import numpy as np
import healpy as hp

# HiPS coverage level (same lC used in the pipeline)
level_coverage = 8

# Open the FITS file and read the HEALPix counts table
with fits.open("LSST_DP1/densmap_o8.fits") as hdul:
    # Convert the binary table to a pandas DataFrame for easy inspection
    df = pd.DataFrame(np.array(hdul[1].data))

# Display the first few rows
print("\nFirst rows of densmap table:")
display(df.head(10))

# Extract the HEALPix counts
counts = df["VALUE"].to_numpy()

# Select non-empty pixels (where count > 0)
ipix = np.flatnonzero(counts > 0)

# Compute nside and npix for the given level
nside = 1 << level_coverage
npix = hp.nside2npix(nside)

# Area of a single pixel (in steradians)
omega_pix = 4.0 * np.pi / npix

# Compute total covered area (steradians and square degrees)
area_sr = len(ipix) * omega_pix
area_deg2 = area_sr * (180.0 / np.pi) ** 2

# Print results
print(f"Number of non-empty pixels: {len(ipix)}")
print(f"Pixel area (sr): {omega_pix:.3e}")
print(f"Total area: {area_deg2:.2f} deg²")

In [ ]:
from mocpy import MOC
import numpy as np
import healpy as hp
import matplotlib.pyplot as plt

# Load the MOC from the FITS file
moc = MOC.from_fits("LSST_DP1/Moc.fits")

# Define the maximum order used to build the binary sky map
order = moc.max_order
nside = 1 << order
npix = hp.nside2npix(nside)

# Depending on your mocpy version, the method name may vary
if hasattr(moc, "flatten_to_order"):
    ipix = moc.flatten_to_order(order)
elif hasattr(moc, "flatten"):
    ipix = moc.flatten()  # older versions don't take an order argument
else:
    raise RuntimeError("Unsupported mocpy version: missing flatten method")

# Create a binary HEALPix map: 1 = inside MOC, 0 = outside
m = np.zeros(npix, dtype=float)
m[ipix] = 1.0

# Display the MOC coverage on a Mollweide projection
hp.mollview(m, nest=True, title=f"MOC (order {order})")
hp.graticule()
plt.show()

# Inspecting any fits file

In [ ]:
from astropy.io import fits
import pandas as pd
import numpy as np

# Path to the FITS file
path = "LSST_DP1/Moc.fits"

# Open the FITS file
with fits.open(path) as hdul:
    # Display general information about the HDUs
    hdul.info()

    # Inspect the columns of the main binary table (HDU 1)
    cols = hdul[1].columns
    print("\nColumns:")
    print(cols)

    # Convert the binary table (HDU 1) into a pandas DataFrame
    df = pd.DataFrame(np.array(hdul[1].data))

# Display the first few rows of the MOC table
print("\nFirst rows of file:")
display(df.head(10))